In [1]:
import pandas as pd
#from optparse import OptionParser
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
#from pybedtools import BedTool
#import sys

In [122]:
def read_hapi_genotypes(file):
    df = pd.read_csv(file)
    df = df[df["Unnamed: 0"].str.contains("Super")]
    df["start"] = df.apply(lambda r: int(r["Unnamed: 0"].split("_")[-1]), axis=1)
    #df["end"] = [df.iloc[i+1]["start"] if i<df.index[-1] else np.nan for i,r in df.iterrows()]
    ndf = df[[c for c in df.columns if "-" in c or c in ["start"]]]
    ndf = ndf.replace("0", np.nan).replace("A", 1).replace("a", np.nan).replace("B", 2).replace("b", np.nan)
    return df, ndf

def scan_phase_changes(hapi, parents):
    
    df_complete = hapi[parents + ["start"]]
    df_informative = df_complete[df_complete.apply(lambda x: "A" in x.values or "B" in x.values, axis=1)]
    df_informative = df_informative.replace("A",2).replace("B",1).replace("0",np.nan)
    df_nomiss = df_informative.dropna()
    
    indexes = df_nomiss.index
    ph = {chrom:[] for chrom in parents}

    for i in range(0, len(indexes)-1):

        i1 = indexes[i]
        i2 = indexes[i+1]
        window = df_nomiss.loc[i1:i2]

        phase_changes = [set(window[chrom].values)=={1,2} for chrom in parents]
        if len(set(phase_changes))==1:
            continue
        
        # Distance between informative sites is >200kb, skip
        #if (window.start.max() - window.start.min()) > 200e3:
        #    continue

        minority_bool = True if phase_changes.count(True)<=2 else False
        minority_parents = [p for p,b in zip(parents, phase_changes) if b==minority_bool]
        for chrom in minority_parents:
            ph[chrom].append([i,i1,i2])
    
    #clean_crossovers(crossovers, df_complete)
    return ph, df_informative, df_nomiss

def get_crossovers(phase_changes, df_polish, indiv):

    crossovers = []
    
    # Group changes of phase in a row, if changes are not odd
    block = []
    groups = []
    min_info_markers = 10
    
    for i,change in enumerate(phase_changes):
        df_loc, start, end = change
        if i==0:
            prev = df_loc
            if prev >= min_info_markers:
                block.append(change)
        else:
            if df_loc-prev <= min_info_markers:
                block.append(change)
            else:
                groups.append(block)
                block = [change]
            prev = df_loc

    groups.append(block)

    
    for block in groups:
        # Crossovers are only odd changes of phase
        if len(block)%2!=0:
            
            # Try to narrow down the crossover, if only 1 clear phase change
            if len(block)==1:
                itr, start, end = block[0]
                refine = df_polish.loc[start:end][[indiv,"start"]].dropna()
                refine = refine#.reset_index() 
                for absi,(i,r) in enumerate(refine.iterrows()):
                    if absi==0:
                        phase = r[indiv]
                    else:
                        if r[indiv]!=phase:
                            cross_start = refine.iloc[absi-1]["start"]
                            cross_end   = refine.iloc[absi]["start"]
                            break
                        phase = r[indiv]
                # If the template is staying in phase, but the other changing
                if len(set(refine[indiv].dropna().values))==1:
                    cross_start = refine.start.min()
                    cross_end = refine.start.max()
                    
                crossovers.append([cross_start, cross_end, i-1, i])
            
            # If multiple (but odd) changes of chase, get narrower range possible
            else:
                start = block[0][1]
                end   = block[-1][-1]
                cross_start = df_polish.loc[start]["start"]
                cross_end   = df_polish.loc[end]["start"]
                crossovers.append([cross_start, cross_end, start, end])

    return crossovers

In [100]:
scaf = "Super_scaffold_345"
fam = "fam2"
famcomplex = "iv-FAMILY02-EG431-EG284"
#hapi_dir = ".."
df,ndf = read_hapi_genotypes("hapi_output/{}.{}/{}.01.csv".format(scaf, fam, famcomplex))

In [101]:
maternal = [c for c in ndf.columns if "M" in c]
paternal = [c for c in ndf.columns if "P" in c]
crossovers = {}

# Detect changes of phase and potential crossovers
for parents in [maternal]:
    ph_change, df_informative = scan_phase_changes(df, parents)
    for p in parents:
        c = get_crossovers(ph_change[p], df_informative, p)
        crossovers[p] = c


[[2203, 3175, 3177], [2204, 3177, 3179]]
[[4841, 6904, 6906], [4842, 6906, 6907]]
[[15695, 33173, 33175], [15696, 33175, 33177]]
[[17765, 37261, 37262]]
[[19077, 39322, 39323], [19079, 39324, 39325]]
[[23076, 45116, 45119], [23078, 45120, 45122]]
[[28863, 53298, 53299], [28864, 53299, 53300]]
[[61287, 112594, 112595], [61288, 112595, 112600]]
[[69614, 136932, 136933], [69615, 136933, 136934]]
[[69640, 136976, 136977], [69641, 136977, 136978]]
[[84358, 175051, 175052], [84359, 175052, 175053]]
[[84901, 175804, 175806], [84902, 175806, 175807]]
[[85714, 176971, 176972], [85715, 176972, 176973]]
[[89850, 185118, 185119], [89851, 185119, 185122]]
[[309, 451, 452], [310, 452, 455]]
[[3065, 4375, 4376], [3066, 4376, 4377]]
[[4305, 6160, 6161], [4306, 6161, 6164]]
[[4563, 6518, 6519], [4564, 6519, 6522]]
[[5382, 7690, 7692], [5385, 7694, 7696]]
[[9550, 14706, 14707], [9553, 14709, 14730]]
[[10149, 16683, 16688], [10150, 16688, 16696]]
[[11484, 20489, 20490], [11485, 20490, 20491]]
[[13028, 25

ValueError: 29477 is not in list

In [102]:
ph_change, df_informative = scan_phase_changes(df, maternal)

In [154]:
def get_crossovers(phase_changes, df_polish, indiv):

    crossovers = []
    
    # Group changes of phase in a row, if changes are not odd
    block = []
    groups = []
    min_info_markers = 10
    
    for i,change in enumerate(phase_changes):
        df_loc, start, end = change
        if i==0:
            prev = df_loc
            if prev >= min_info_markers:
                block.append(change)
        else:
            if df_loc-prev <= min_info_markers:
                block.append(change)
            else:
                groups.append(block)
                block = [change]
            prev = df_loc

    groups.append(block)
    if len(phase_changes)==2:
        groups = []
        
    for block in groups:
        # Crossovers are only odd changes of phase
        if len(block)%2!=0:
            
            # Try to narrow down the crossover, if only 1 clear phase change
            if len(block)==1:
                itr, start, end = block[0]
                refine = df_polish.loc[start:end][[indiv,"start"]].dropna()
                refine = refine#.reset_index() 
                for absi,(i,r) in enumerate(refine.iterrows()):
                    if absi==0:
                        phase = r[indiv]
                    else:
                        if r[indiv]!=phase:
                            cross_start = refine.iloc[absi-1]["start"]
                            cross_end   = refine.iloc[absi]["start"]
                            break
                        phase = r[indiv]
                # If the template is staying in phase, but the other changing
                if len(set(refine[indiv].dropna().values))==1:
                    cross_start = refine.start.min()
                    cross_end = refine.start.max()
                    
                crossovers.append([cross_start, cross_end, start, end])
            
            # If multiple (but odd) changes of chase, get narrower range possible
            else:
                start = block[0][1]
                end   = block[-1][-1]
                cross_start = df_polish.loc[start]["start"]
                cross_end   = df_polish.loc[end]["start"]
                crossovers.append([cross_start, cross_end, start, end])
    
    index_list = list(df_informative.index)
    maxl = len(index_list)
    info_markers_limit = 30
    
    clean_crossovers = []
    for c in crossovers:
        print(c)
        istart = index_list.index(c[2])
        iend = index_list.index(c[3])
        if istart<=info_markers_limit or iend>=(maxl-info_markers_limit):
            continue
        clean_crossovers.append(c)
            
    
    return clean_crossovers

def scan_phase_changes(hapi, parents):
    
    df_complete = hapi[parents + ["start"]]
    df_informative = df_complete[df_complete.apply(lambda x: "A" in x.values or "B" in x.values, axis=1)]
    df_informative = df_informative.replace("A",2).replace("B",1).replace("0",np.nan)
    df_nomiss = df_informative.dropna()
    
    indexes = df_nomiss.index
    ph = {chrom:[] for chrom in parents}

    for i in range(0, len(indexes)-1):

        i1 = indexes[i]
        i2 = indexes[i+1]
        window = df_nomiss.loc[i1:i2]

        phase_changes = [set(window[chrom].values)=={1,2} for chrom in parents]
        if len(set(phase_changes))==1:
            continue
        
        # Distance between informative sites is >200kb, skip
        #if (window.start.max() - window.start.min()) > 200e3:
        #    continue

        minority_bool = True if phase_changes.count(True)<=2 else False
        minority_parents = [p for p,b in zip(parents, phase_changes) if b==minority_bool]
        for chrom in minority_parents:
            ph[chrom].append([i,i1,i2])
    
    #clean_crossovers(crossovers, df_complete)
    return ph, df_informative, df_nomiss, indexes

In [143]:
indexes2 = list(df_nomiss.index)

In [155]:
p = 'M-EG431-284-472'
#print(ph_change[p])
ph_change, df_informative, df_nomiss, indexes = scan_phase_changes(df, maternal)
get_crossovers(ph_change[p], df_informative, p)

[11590941.0, 11591784.0, 29475, 29478]


[[11590941.0, 11591784.0, 29475, 29478]]